In [4]:
import os
import cv2
from tqdm import tqdm

# Ruta base donde se encuentran los datos
base_path = r'C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas'

def process_folder_to_video(input_folder, output_video_path, fps=30):
    """
    Convierte todas las imágenes (png, jpg, jpeg) en un directorio a un video MP4
    
    Args:
        input_folder: Ruta de la carpeta con las imágenes
        output_video_path: Ruta para guardar el video generado
        fps: Frames por segundo para el video (default: 30)
    
    Returns:
        bool: True si el video se generó correctamente, False en caso contrario
    """
    # Obtener todas las imágenes en la carpeta, ordenadas
    image_files = sorted([f for f in os.listdir(input_folder) 
                      if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
    
    # Verificar si hay imágenes
    if not image_files:
        print(f"❌ No se encontraron imágenes válidas en la carpeta: {input_folder}")
        return False
    
    # Leer la primera imagen para obtener las dimensiones
    first_image_path = os.path.join(input_folder, image_files[0])
    frame = cv2.imread(first_image_path)
    
    # Comprobar si la imagen se cargó correctamente
    if frame is None:
        print(f"❌ No se pudo leer la imagen: {first_image_path}")
        return False
    
    height, width = frame.shape[:2]
    
    # Configurar el video
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Formato de compresión para video mp4
    
    # Asegurarse de que exista el directorio para guardar el video
    os.makedirs(os.path.dirname(output_video_path), exist_ok=True)
    
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    # Añadir las imágenes al video
    for filename in tqdm(image_files, desc=f"Procesando {os.path.basename(input_folder)}"):
        image_path = os.path.join(input_folder, filename)
        frame = cv2.imread(image_path)
        
        # Verificar si la imagen fue leída correctamente
        if frame is None:
            print(f"⚠️ Imagen no leída: {image_path}")
            continue
        
        # Añadir la imagen al video
        out.write(frame)
    
    # Liberar el objeto de salida
    out.release()
    print(f"✅ Video generado correctamente en: {output_video_path}")
    return True

def process_all_a_ver_folders():
    """
    Recorre todas las carpetas 'A ver' dentro de la estructura y procesa las imágenes RGB
    para convertirlas en videos, guardando cada video en su carpeta de muestra correspondiente
    """
    # Construir la ruta base para 'A ver'
    a_ver_base = os.path.join(base_path, 'A ver')
    
    # Verificar si la carpeta existe
    if not os.path.exists(a_ver_base):
        print(f"❌ La carpeta 'A ver' no existe en: {base_path}")
        return
    
    # Obtener todas las subcarpetas 'muestra_X'
    muestra_folders = [f for f in os.listdir(a_ver_base) if f.startswith('muestra_')]
    
    if not muestra_folders:
        print(f"❌ No se encontraron carpetas 'muestra_X' en: {a_ver_base}")
        return
    
    videos_generated = 0
    
    # Procesar cada carpeta muestra
    for muestra_folder in muestra_folders:
        muestra_path = os.path.join(a_ver_base, muestra_folder)
        
        # Buscar la carpeta 'rgb' dentro de cada muestra
        rgb_path = os.path.join(muestra_path, 'CARA')
        
        if os.path.exists(rgb_path) and os.path.isdir(rgb_path):
            # Generar nombre del video de salida dentro de la carpeta muestra
            output_video_name = f"video_{muestra_folder}_rgb.mp4"
            output_path = os.path.join(muestra_path, output_video_name)
            
            # Procesar carpeta
            success = process_folder_to_video(rgb_path, output_path)
            if success:
                videos_generated += 1
        else:
            print(f"⚠️ No se encontró la carpeta 'rgb' en: {muestra_path}")
    
    print(f"\n✅ Proceso completado. Videos generados: {videos_generated}")

def process_all_expressions():
    """
    Recorre todas las carpetas de expresiones (incluyendo 'A ver', 'Feliz', etc.)
    y procesa las imágenes RGB para convertirlas en videos, guardando cada video
    en su carpeta de muestra correspondiente
    """
    # Obtener todas las carpetas de expresiones
    expression_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f))]
    
    if not expression_folders:
        print(f"❌ No se encontraron carpetas de expresiones en: {base_path}")
        return
    
    videos_generated = 0
    
    # Procesar cada carpeta de expresión
    for expression in expression_folders:
        expression_path = os.path.join(base_path, expression)
        print(f"\n🔍 Procesando expresión: {expression}")
        
        # Obtener todas las subcarpetas 'muestra_X'
        try:
            muestra_folders = [f for f in os.listdir(expression_path) if f.startswith('muestra_')]
        except Exception as e:
            print(f"❌ Error al listar carpetas en {expression_path}: {e}")
            continue
        
        for muestra_folder in muestra_folders:
            muestra_path = os.path.join(expression_path, muestra_folder)
            
            # Buscar la carpeta 'rgb' dentro de cada muestra
            rgb_path = os.path.join(muestra_path, 'CARA')#AQUI DECLARAS QUE CARPETAS USAR PARA HACER EL VIDEO
            
            if os.path.exists(rgb_path) and os.path.isdir(rgb_path):
                # Generar nombre del video de salida dentro de la carpeta muestra
                safe_expression = expression.replace(' ', '_').lower()
                output_video_name = f"{safe_expression}_{muestra_folder}.mp4"
                output_path = os.path.join(muestra_path, output_video_name)
                
                # Procesar carpeta
                success = process_folder_to_video(rgb_path, output_path)
                if success:
                    videos_generated += 1
            else:
                print(f"⚠️ No se encontró la carpeta 'rgb' en: {muestra_path}")
    
    print(f"\n✅ Proceso completado. Videos generados: {videos_generated}")

def check_generated_videos():
    """
    Verifica los videos generados buscando archivos MP4 en todas las carpetas
    de muestras dentro de las expresiones
    """
    videos_found = []
    
    # Recorrer todas las carpetas de expresiones
    for expression in os.listdir(base_path):
        expression_path = os.path.join(base_path, expression)
        
        if not os.path.isdir(expression_path):
            continue
            
        # Recorrer las carpetas de muestras
        try:
            for muestra_folder in os.listdir(expression_path):
                muestra_path = os.path.join(expression_path, muestra_folder)
                
                if not os.path.isdir(muestra_path) or not muestra_folder.startswith('muestra_'):
                    continue
                    
                # Buscar archivos MP4 en la carpeta de muestra
                for file in os.listdir(muestra_path):
                    if file.endswith('.mp4'):
                        video_path = os.path.join(muestra_path, file)
                        videos_found.append((expression, muestra_folder, file, video_path))
        except Exception as e:
            print(f"Error al explorar {expression_path}: {e}")
    
    if not videos_found:
        print("❌ No se encontraron videos generados.")
        return
    
    print(f"📊 Videos generados ({len(videos_found)}):\n")
    
    for i, (expression, muestra, video, video_path) in enumerate(videos_found, 1):
        video_size_mb = os.path.getsize(video_path) / (1024 * 1024)
        
        # Obtener información del video usando OpenCV
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"{i}. {expression}/{muestra}/{video} - ⚠️ No se pudo abrir el video")
            continue
            
        # Obtener propiedades
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        duration = frame_count / fps if fps > 0 else 0
        
        cap.release()
        
        print(f"{i}. {expression}/{muestra}/{video}")
        print(f"   - Tamaño: {video_size_mb:.2f} MB")
        print(f"   - Resolución: {width}x{height}")
        print(f"   - FPS: {fps:.2f}")
        print(f"   - Frames: {frame_count}")
        print(f"   - Duración: {duration:.2f} segundos\n")

# Ejecutar el procesamiento
if __name__ == "__main__":
    print("¿Qué deseas procesar?")
    print("1. Solo las carpetas de 'A ver'")
    print("2. Todas las expresiones (incluyendo 'A ver' y otras)")
    print("3. Solo verificar videos existentes")
    
    choice = input("Ingresa tu opción (1, 2 o 3): ")
    
    if choice == "1":
        print("\nProcesando solo las carpetas de 'A ver'...\n")
        process_all_a_ver_folders()#Se puede modificar
    elif choice == "2":
        print("\nProcesando todas las expresiones...\n")
        process_all_expressions()
    elif choice == "3":
        print("\nVerificando videos existentes...\n")
    else:
        print("❌ Opción inválida. Saliendo.")
        exit()
    
    # Verificar los videos generados
    check_generated_videos()

¿Qué deseas procesar?
1. Solo las carpetas de 'A ver'
2. Todas las expresiones (incluyendo 'A ver' y otras)
3. Solo verificar videos existentes

Procesando todas las expresiones...


🔍 Procesando expresión: A ver


Procesando CARA: 100%|██████████| 69/69 [00:00<00:00, 125.43it/s]


✅ Video generado correctamente en: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_1\a_ver_muestra_1.mp4


Procesando CARA: 100%|██████████| 70/70 [00:00<00:00, 123.41it/s]


✅ Video generado correctamente en: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\A ver\muestra_2\a_ver_muestra_2.mp4


...

✅ Video generado correctamente en: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\Triste\muestra_4\triste_muestra_4.mp4


Procesando CARA: 100%|██████████| 69/69 [00:00<00:00, 127.79it/s]


✅ Video generado correctamente en: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\Triste\muestra_5\triste_muestra_5.mp4


Procesando CARA: 100%|██████████| 84/84 [00:00<00:00, 126.52it/s]


✅ Video generado correctamente en: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 2\senas_procesadas\Triste\muestra_6\triste_muestra_6.mp4

✅ Proceso completado. Videos generados: 83
📊 Videos generados (172):

1. A ver/muestra_1/a_ver_muestra_1.mp4
   - Tamaño: 0.70 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 69
   - Duración: 2.30 segundos

2. A ver/muestra_1/video_a_ver_muestra_1.mp4
   - Tamaño: 0.40 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 69
   - Duración: 2.30 segundos

3. A ver/muestra_1/video_muestra_1_rgb.mp4
   - Tamaño: 0.70 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 69
   - Duración: 2.30 segundos

4. A ver/muestra_2/a_ver_muestra_2.mp4
   - Tamaño: 0.72 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 70
   - Duración: 2.33 segundos

5. A ver/muestra_2/video_a_ver_muestra_2.mp4
   - Tamaño: 0.42 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 70
   - Duración: 2.33 segundos

6. A

In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm

def process_folder_to_video(input_folder, output_video_path, fps=30):
    """
    Convierte todas las imágenes (png, jpg, jpeg) en un directorio a un video MP4
    
    Args:
        input_folder: Ruta de la carpeta con las imágenes
        output_video_path: Ruta para guardar el video generado
        fps: Frames por segundo para el video (default: 30)
    
    Returns:
        bool: True si el video se generó correctamente, False en caso contrario
    """
    try:
        # Obtener todas las imágenes en la carpeta, ordenadas
        image_files = sorted([f for f in os.listdir(input_folder) 
                          if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        
        # Verificar si hay imágenes
        if not image_files:
            print(f"❌ No se encontraron imágenes válidas en la carpeta: {input_folder}")
            return False
        
        # Leer la primera imagen para obtener las dimensiones
        first_image_path = os.path.join(input_folder, image_files[0])
        
        # Leer imagen usando numpy para manejar rutas con espacios
        try:
            # Método 1: Lectura directa
            frame = cv2.imread(first_image_path)
            
            # Si falla, intentar método alternativo con numpy
            if frame is None:
                img_data = np.fromfile(first_image_path, dtype=np.uint8)
                frame = cv2.imdecode(img_data, cv2.IMREAD_COLOR)
        except Exception as e:
            print(f"❌ Error al leer la primera imagen {first_image_path}: {e}")
            return False
        
        # Comprobar si la imagen se cargó correctamente
        if frame is None:
            print(f"❌ No se pudo leer la imagen: {first_image_path}")
            return False
        
        height, width = frame.shape[:2]
        
        # Configurar el video
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Formato de compresión para video mp4
        
        # Asegurarse de que exista el directorio para guardar el video
        os.makedirs(os.path.dirname(output_video_path), exist_ok=True)
        
        out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
        
        # Añadir las imágenes al video
        print(f"Procesando {len(image_files)} imágenes de {os.path.basename(input_folder)}")
        
        successful_frames = 0
        failed_frames = 0
        
        for filename in tqdm(image_files, desc="Generando video"):
            image_path = os.path.join(input_folder, filename)
            
            try:
                # Intentar primero lectura directa
                frame = cv2.imread(image_path)
                
                # Si falla, intentar con numpy para manejar espacios
                if frame is None:
                    img_data = np.fromfile(image_path, dtype=np.uint8)
                    frame = cv2.imdecode(img_data, cv2.IMREAD_COLOR)
                
                # Verificar si la imagen fue leída correctamente
                if frame is None:
                    print(f"⚠️ Imagen no leída: {image_path}")
                    failed_frames += 1
                    continue
                
                # Añadir la imagen al video
                out.write(frame)
                successful_frames += 1
                
            except Exception as e:
                print(f"❌ Error procesando {image_path}: {e}")
                failed_frames += 1
        
        # Liberar el objeto de salida
        out.release()
        
        # Resumen del proceso
        if successful_frames > 0:
            print(f"✅ Video generado correctamente en: {output_video_path}")
            print(f"   - Frames exitosos: {successful_frames}")
            if failed_frames > 0:
                print(f"   - Frames fallidos: {failed_frames}")
            return True
        else:
            print(f"❌ No se pudo generar el video. Todos los frames fallaron.")
            return False
            
    except Exception as e:
        print(f"❌ Error general durante el procesamiento: {e}")
        return False

def check_video_info(video_path):
    """
    Verifica y muestra información de un video
    
    Args:
        video_path: Ruta del video a verificar
    """
    if not os.path.exists(video_path):
        print(f"❌ El video no existe: {video_path}")
        return
        
    video_size_mb = os.path.getsize(video_path) / (1024 * 1024)
    
    # Obtener información del video usando OpenCV
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"⚠️ No se pudo abrir el video: {video_path}")
        return
        
    # Obtener propiedades
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    duration = frame_count / fps if fps > 0 else 0
    
    cap.release()
    
    print(f"\n📊 Información del video generado:")
    print(f"   - Ruta: {video_path}")
    print(f"   - Tamaño: {video_size_mb:.2f} MB")
    print(f"   - Resolución: {width}x{height}")
    print(f"   - FPS: {fps:.2f}")
    print(f"   - Frames: {frame_count}")
    print(f"   - Duración: {duration:.2f} segundos")

# Menú principal
if __name__ == "__main__":
    print("===== GENERADOR DE VIDEOS A PARTIR DE IMÁGENES =====")
    print("Este script convertirá una secuencia de imágenes en un video MP4.\n")
    
    # Ruta de la carpeta con las imágenes (frames)
    input_folder = r"C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_1\salida"  # MODIFICA ESTA RUTA
    
    # Ruta donde guardar el video generado
    output_video = r"C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_1\video_salida.mp4"  # MODIFICA ESTA RUTA
    
    # Definir los FPS del video
    fps = 30  # Frames por segundo (puedes modificar este valor)
    
    # Mostrar la configuración
    print(f"Configuración:")
    print(f"- Carpeta de entrada (frames): {input_folder}")
    print(f"- Archivo de video de salida: {output_video}")
    print(f"- FPS (frames por segundo): {fps}")
    
    # Verificar si las rutas son válidas
    if not os.path.exists(input_folder):
        print(f"\n❌ Error: La carpeta de entrada no existe: {input_folder}")
        print("Por favor, modifica la variable 'input_folder' en el código.")
        exit(1)
        
    if not os.path.isdir(input_folder):
        print(f"\n❌ Error: La ruta de entrada no es una carpeta: {input_folder}")
        print("Por favor, proporciona una ruta válida a una carpeta.")
        exit(1)
    
    # Verificar si la carpeta de salida existe
    output_dir = os.path.dirname(output_video)
    if output_dir and not os.path.exists(output_dir):
        try:
            os.makedirs(output_dir, exist_ok=True)
            print(f"\nℹ️ Se creó la carpeta de salida: {output_dir}")
        except Exception as e:
            print(f"\n❌ Error al crear la carpeta de salida: {e}")
            print("Por favor, proporciona una ruta de salida válida.")
            exit(1)
    
    # Procesar la carpeta para generar el video
    print("\nIniciando proceso de generación de video...")
    success = process_folder_to_video(input_folder, output_video, fps)
    
    # Verificar el video generado si fue exitoso
    if success:
        check_video_info(output_video)

===== GENERADOR DE VIDEOS A PARTIR DE IMÁGENES =====
Este script convertirá una secuencia de imágenes en un video MP4.

Configuración:
- Carpeta de entrada (frames): C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_1\salida
- Archivo de video de salida: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_1\video_salida.mp4
- FPS (frames por segundo): 30

Iniciando proceso de generación de video...
Procesando 36 imágenes de salida


Generando video: 100%|██████████| 36/36 [00:00<00:00, 118.69it/s]

✅ Video generado correctamente en: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_1\video_salida.mp4
   - Frames exitosos: 36

📊 Información del video generado:
   - Ruta: C:\Users\Invitado\Documents\facial_expression_detector\data_procesada\data\SUJETO 3\señas_procesadas\A ver\muestra_1\video_salida.mp4
   - Tamaño: 0.33 MB
   - Resolución: 640x640
   - FPS: 30.00
   - Frames: 36
   - Duración: 1.20 segundos
